1 A

In [1]:
import pickle 

def saveDictionary(tmp_dict):
    
    try:
        with open('words.txt', 'w') as dictionary:
            pickle.dump(tmp_dict, dictionary)
        print("SUCCESS: writing to dictionary")
    except Exception as e:
        print(f"ERROR: {e}")


def getDictionary():
   
    try:
        with open('words.txt', 'r') as dictionary:
            tmp_dict = pickle.load(dictionary)
        print("SUCCESS: reading from dictionary")
        return tmp_dict
    except Exception as e:
        print(f"ERROR: {e}")

def addNewWord():

    print("__________")
    print("ADD NEW WORD:")
    print("__________")
    print("Enter a Word to add")
    word = input(">>")
    print(f"Enter a meaning of {word} to add")
    meaning = input(">>")
    dictionary = getDictionary()
    dictionary[word] = meaning
    saveDictionary(dictionary)

    main_menu()


def findMeaning():
    
    print("__________")
    print("FIND WORD:")
    print("__________")
    print("Enter a Word to find")
    word = input(">>")
    dictionary = getDictionary()
    if word in dictionary:
        print(f"Found Entry: [{word} : {dictionary[word]}]")
    else:
        print("ERROR: The word doesn't exist!")
        findMeaning()

    main_menu()


def updateWord():
    print("__________")
    print("UPDATE WORD:")
    print("__________")
    print("Enter an old Word to edit")
    word = input(">>")
    dictionary = getDictionary()
    if word in dictionary:
        print(f"Enter a New Meaning to edit [{word} : {dictionary[word]}] to:")
        new_meaning = input(">>")
        dictionary[word] = new_meaning
        saveDictionary(dictionary)
        print(f"SUCCESS: Updated to [{word} : {dictionary[word]}]")
    else:
        print("ERROR: The word doesn't exist! impossible to update!")
        updateWord()
        
    main_menu()


def main_menu():
    print("__________")
    print("ENGLISH DICTIONARY APP")
    print("__________")
    print("1. Add a new word")
    print("2. Find the meaning")
    print("3. Update a word")
    print("4. Exit")
    print("__________")
    
    n = int(input(">>"))
    if n==1:
        addNewWord()
    elif n==2:
        findMeaning()
    elif n==3:
        updateWord()
    elif n==4:
        exit()
    else:
        
        print("ERROR: Please Enter a digit according to Menu.")
        main_menu()
main_menu()

__________
ENGLISH DICTIONARY APP
__________
1. Add a new word
2. Find the meaning
3. Update a word
4. Exit
__________
>>4


1B

In [16]:
import sqlite3                    
from tkinter import *             
from tkinter import messagebox


class DB:                          
    def __init__(self):          
        self.conn = sqlite3.connect("mybooks.db")  
        self.cur = self.conn.cursor()    
        self.cur.execute(             
            "CREATE TABLE IF NOT EXISTS book (id INTEGER PRIMARY KEY, title TEXT, author TEXT, year TEXT)")  
        self.conn.commit()  

    def __del__(self):          
        self.conn.close()   

    def view(self):         
        self.cur.execute("SELECT * FROM book") 
        rows = self.cur.fetchall() 
        return rows

    def insert(self, title, author, isbn):  
        self.cur.execute("INSERT INTO book VALUES (NULL,?,?,?)", (title, author, isbn,))
        self.conn.commit()
        self.view()

    def update(self, id, title, author):    
        self.cur.execute("UPDATE book SET title=?, author=? WHERE id=?", (title, author, id,))
        self.conn.commit()
        self.view()

    def delete(self, id):                   
        self.cur.execute("DELETE FROM book WHERE id=?", (id,))
        self.conn.commit()
        self.view()

    def search(self, title="", author=""):  
        self.cur.execute("SELECT * FROM book WHERE title=? OR author=?", (title, author,))
        rows = self.cur.fetchall()
        return rows


db = DB()  

def get_selected_row(event): 
    global selected_tuple
    index = list1.curselection()[0] 
    selected_tuple = list1.get(index) 
    e1.delete(0, END)                
    e1.insert(END, selected_tuple[1]) 
    e2.delete(0, END)
    e2.insert(END, selected_tuple[2]) 
    e3.delete(0, END)
    e3.insert(END, selected_tuple[3]) 


def view_command():          
    list1.delete(0, END)   
    for row in db.view():   
        list1.insert(END, row)  


def search_command():       
    list1.delete(0, END)   
    for row in db.search(title_text.get(), author_text.get()): 
        list1.insert(END, row) 


def add_command():          
    db.insert(title_text.get(), author_text.get(), isbn_text.get())  
    list1.delete(0, END) 
    list1.insert(END, (title_text.get(), author_text.get(), isbn_text.get())) 


def delete_command():  
    db.delete(selected_tuple[0]) 


def update_command():
    db.update(selected_tuple[0], title_text.get(), author_text.get()) 


window = Tk() 

window.title("A Library Application!") 


def on_closing(): 
    dd = db
    if messagebox.askokcancel("Quit", "Do you want to quit?"): 
        window.destroy()
        del dd 


window.protocol("WM_DELETE_WINDOW", on_closing)  

l1 = Label(window, text="Book Title") 
l1.grid(row=0, column=0) 

l2 = Label(window, text="Author")
l2.grid(row=0, column=2)

l3 = Label(window, text="Year")
l3.grid(row=1, column=0)

title_text = StringVar()
e1 = Entry(window, textvariable=title_text)
e1.grid(row=0, column=1)

author_text = StringVar() 
e2 = Entry(window, textvariable=author_text)
e2.grid(row=0, column=3)

isbn_text = StringVar() 
e3 = Entry(window, textvariable=isbn_text)
e3.grid(row=1, column=1)

list1 = Listbox(window, height=25, width=65) 
list1.grid(row=2, column=0, rowspan=6, columnspan=2)

sb1 = Scrollbar(window) 
sb1.grid(row=2, column=2, rowspan=6)

list1.configure(yscrollcommand=sb1.set) 
sb1.configure(command=list1.yview)

list1.bind('<<ListboxSelect>>', get_selected_row)

b1 = Button(window, text="View all", width=12, command=view_command)  
b1.grid(row=2, column=3) 

b2 = Button(window, text="Search entry", width=12, command=search_command)
b2.grid(row=3, column=3)

b3 = Button(window, text="Add entry", width=12, command=add_command)
b3.grid(row=4, column=3)

b4 = Button(window, text="Update selected", width=12, command=update_command)
b4.grid(row=5, column=3)

b5 = Button(window, text="Delete selected", width=12, command=delete_command)
b5.grid(row=6, column=3)

b6 = Button(window, text="Close", width=12, command=window.destroy)
b6.grid(row=7, column=3)

window.mainloop() 